In [1]:
import numpy as np 
import pandas as pd 


In [41]:
df=pd.read_csv(r'covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [43]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [45]:
df['gender'].value_counts()

gender
Female    59
Male      41
Name: count, dtype: int64

In [47]:
df.isna().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [49]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [51]:
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split

In [53]:
le=LabelEncoder()
df['has_covid']=le.fit_transform(df['has_covid'])

In [55]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,0
1,27,Male,100.0,Mild,Delhi,1
2,42,Male,101.0,Mild,Delhi,0
3,31,Female,98.0,Mild,Kolkata,0
4,65,Female,101.0,Mild,Mumbai,0
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,0
96,51,Female,101.0,Strong,Kolkata,1
97,20,Female,101.0,Mild,Bangalore,0
98,5,Female,98.0,Strong,Mumbai,0


In [57]:
oe=OrdinalEncoder(categories=[['Mild','Strong']],dtype=np.int32)

In [59]:
df['cough']=oe.fit_transform(df[['cough']])
df.sample(5)

,age,gender,fever,cough,city,has_covid
85,16,Female,103.0,0,Bangalore,1
68,54,Female,104.0,1,Kolkata,0
38,49,Female,101.0,0,Delhi,1
8,19,Female,100.0,1,Bangalore,0
32,34,Female,101.0,1,Delhi,1


In [61]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2,random_state=42)

In [75]:
# impute fever column using SimpleImputer 
from sklearn.impute import SimpleImputer
X_train.shape

(80, 5)

In [67]:
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.fit_transform(X_test[['fever']])

In [77]:
X_train_fever

array([[101.],
       [100.],
       [100.],
       [100.],
       [103.],
       [103.],
       [102.],
       [101.],
       [101.],
       [101.],
       [ 98.],
       [104.],
       [103.],
       [104.],
       [100.],
       [101.],
       [104.],
       [102.],
       [102.],
       [103.],
       [104.],
       [102.],
       [101.],
       [104.],
       [102.],
       [101.],
       [ 99.],
       [101.],
       [104.],
       [102.],
       [100.],
       [ 98.],
       [ 98.],
       [101.],
       [100.],
       [100.],
       [101.],
       [104.],
       [101.],
       [103.],
       [101.],
       [ 98.],
       [ 99.],
       [ 99.],
       [101.],
       [ 99.],
       [101.],
       [104.],
       [ 98.],
       [101.],
       [103.],
       [101.],
       [ 98.],
       [ 99.],
       [ 98.],
       [ 99.],
       [102.],
       [101.],
       [101.],
       [104.],
       [100.],
       [ 98.],
       [100.],
       [101.],
       [100.],
       [100.],
       [ 9

In [79]:
# onehotencoding im gender and city 
ohe=OneHotEncoder(sparse_output=False,drop='first')
X_train_gender_city=ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city=ohe.fit_transform(X_test[['gender','city']])

In [100]:
# Extracting age 
X_train_Age=X_train.drop(columns=['gender','fever','cough','city']).values
X_train_Age.shape

(80, 1)

In [102]:
X_test_Age=X_test.drop(columns=['gender','fever','cough','city']).values
X_test_Age.shape

(20, 1)

In [112]:
X_train_Cough=X_train.drop(columns=['gender','fever','age','city']).values
X_train_Cough.shape

(80, 1)

In [104]:
X_test_Cough=X_test.drop(columns=['gender','fever','age','city']).values
X_test_Cough.shape

(20, 1)

In [120]:
X_train_transformed=np.concatenate((X_train_Age,X_train_fever,X_train_gender_city,X_train_Cough),axis=1)
X_train_transformed.shape

(80, 7)

In [118]:
X_test_transformed=np.concatenate((X_test_Age,X_test_fever,X_test_gender_city,X_test_Cough),axis=1)
X_test_transformed.shape

(20, 7)

## Column Transformer 


In [123]:
from sklearn.compose import ColumnTransformer

In [129]:
transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    # ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']), already done 
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [133]:
transformer.fit_transform(X_train).shape

(80, 7)

In [137]:
transformer.transform(X_test).shape

(20, 7)